<p>1) Load all of the needed python packages to run this notebook. These include ODM2API, ipython widgets for user interaction, bokeh for interactive plots. ui_utils </p>


In [1]:
import sys
import os
import datetime
import getpass
import utilities.ui_utils as ui_utils
from bokeh.plotting import  figure, show 
from IPython.display import display, HTML
from odm2api.ODMconnection import dbconnection
import odm2api.ODM2.services.readService as odm2
from odm2api.ODM2.models import *
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource

2) The code below retreives the ODM2 login prompt from ui_utils python code which should be in your utilities folder. This will log directly into SQL.  

In [2]:
#print("Enter your ODM2 username") 
container = ui_utils.ODM2LoginPrompt()
container

3) Next we prompt a user for their password. 

In [3]:
print("enter your password: ")
p = getpass.getpass()

enter your password: 
········


<p> 4) Here we look into the login prompt container. to retrieve the database connection information entered by the user. </p>
<p> 5) On the second line, I'm using the ODM2API read service for retrieving result values this should help make retrieving 
them agnostic to the result type. </p>  

In [2]:
session_factory = dbconnection.createConnection('postgresql', container.children[1].value, 
                                                container.children[2].value, 
                                                container.children[0].value, p) 



read = odm2.ReadODM2(session_factory)

6) Next we retreive another UI element for selecting observation actions. Which we will use to filter results. Otherwise the list of results maybe too long. 

In [3]:
DBSession = session_factory.getSession()
actionWidget, awidget = ui_utils.actionWidget(DBSession)
display(awidget) 


3


7) Now we get a UI element for results, for which we would like to display values. 

In [9]:

#featureaction = 1700
actionid = actionWidget.value
resultWidget, rwidget = ui_utils.resultWidget(read, actionid)
display(rwidget)

16681


8) Now we create a UI element for entering the dates we would like to filter the time series on. 

In [10]:
startdate_enddate_container = ui_utils.startDateEndDateContainer()
startdate_enddate_container

9) Next we read more information about the result, pull the results values for the time frame we specified and create a data dictionary our selection graph can display. 

In [6]:
selectedResult = read.getDetailedResultInfo(resultTypeCV = 'Time series coverage',resultID=resultWidget.value)
#resultValues = read.getResultValues(resultid=resultWidget.value, starttime='2016-8-1', 
#                                                 endtime= '2016-8-30')
resultValues = read.getResultValues(resultid=resultWidget.value, starttime=startdate_enddate_container.children[0].value, 
                                                 endtime= startdate_enddate_container.children[1].value)
data = dict(x=resultValues.ValueDateTime, y= resultValues.DataValue, z=resultValues.ValueID)

10) Now we create a bokeh (graphing package) ColumnDataSource for our graph, create our plot (also using ui_utils) and display it. 

In [7]:
s1 = ColumnDataSource(data=data)
plot = ui_utils.selectPlot(s1,selectedResult)
output_notebook()
show(plot)

7019.6
7028.0


Loading BokehJS ...

11) The code below prints out the selected data points. This can be used for diagnostic purposes and is commented out here. 

In [ ]:
# data = zip([s1.data['x'][i] for i in inds],
#    [s1.data['y'][i] for i in inds],
#          [s1.data['z'][i] for i in inds])
# print(data)

12) Next we prompt a user to enter an annotation and annotation code to prompt the user for. 

In [8]:

annotationcontainer = ui_utils.annotationContainer()
annotationcontainer

13) Using the below drop down list select the data quality code for these records. 

In [21]:

dataqualityWidget, qdwidget =ui_utils.dataqualityWidget(DBSession)
display(qdwidget)

Good


14) This creates the actual annotation database record and the associated result value annotations. This links each result value to the annotation.  

In [25]:
annotation_type =  DBSession.query(CVAnnotationType).filter_by(Name="Time series result value annotation").one() 
author =  DBSession.query(People).filter_by(PersonFirstName="Miguel").one()
citation =  DBSession.query(Citations).filter_by(CitationID=327).one() 
print(annotation_type.Name)
print(author.PersonFirstName)
print(citation.Title)
tsrvavalueids = []
annotation = Annotations(AnnotationTypeCV=annotation_type.Name,
                           AnnotationCode = annotationcontainer.children[0].value,
                           AnnotationText = annotationcontainer.children[1].value,
                           AnnotationDateTime = datetime.datetime.now(),
                           AnnotationUTCOffset = 0,
                           AnnotationLink = 'test',
                           AnnotatorID = 1,
                           CitationID =327)
print(annotation)
DBSession.add(annotation)
DBSession.commit()
annotationid = annotation.AnnotationID
print(annotation)
for i in inds:
    valueid = s1.data['z'][i]
    tsrvAnnotation = TimeSeriesResultValueAnnotations(ValueID = valueid,
                                                     AnnotationID =annotationid)
    DBSession.add(tsrvAnnotation)
    DBSession.commit()
    tsrvavalueids.append(tsrvAnnotation.BridgeID)
for i in inds:
    valueid = s1.data['z'][i]
    tsrv = DBSession.query(TimeSeriesResultValues).filter_by(ValueID=valueid).one()
    tsrv.QualityCodeCV = dataquality_codeWidget.value
DBSession.commit()    

Time series result value annotation
Miguel
Lithological Influences on Contemporary and Long-Term Regolith Weathering at the Luquillo Critical Zone Observatory

Buss, H.L., Lara Chapela M., Moore, O.W., Kurtz A.C., Schulz, M.S., White A.F.


<p> 1) This will last block of code will delete the values back out of the database. </p>

In [23]:

#print(tsrvavalueids)
time_series_values_annotations = DBSession.query(TimeSeriesResultValueAnnotations).filter(
        TimeSeriesResultValueAnnotations.BridgeID.in_(tsrvavalueids))
for time_series_value_annotation in time_series_values_annotations:
    DBSession.delete(time_series_value_annotation)
DBSession.commit()

DBSession.delete(annotation)
DBSession.commit()